# Finding the similarity between title and product text using word2vec and cosine similarity

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')
import unicodedata
from numpy import dot
from numpy.linalg import norm

import cv2
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
test_df = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
DATA_PATH = '/kaggle/input/shopee-product-matching/'
print(gensim.__version__)

# Data preprocessing

In [ ]:
def cleanData(dataParse):
    data = unicodedata.normalize('NFKC', dataParse)
    data = re.sub(r'【.*】', '', data)
    data = re.sub(r'\[.*\]', '', data)
    data = re.sub(r'「.*」', '', data)
    data = re.sub(r'\(.*\)', '', data)
    data = re.sub(r'\<.*\>', '', data)
    data = re.sub(r'[※@◎].*$', '', data)
    return data.lower() #Returns the parsed tweets


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(cleanData(text), pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            if token == 'xxxx':
                continue
            result.append(lemmatize_stemming(token))
    
    return result

In [ ]:
processed_docs = train_df['title'].map(preprocess)
processed_docs =list(processed_docs)

In [ ]:
processed_docs[:5] # clean document

# Load model

In [ ]:
w2v_model = Word2Vec.load("/kaggle/input/train-model/word2vec_model.model")

# Find the similarity between two vector

In [ ]:
# Generate the average word2vec for the each title description

def vectors_test(): #test_df
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings_test
    word_embeddings_test = []
    test_processed_docs = test_df['title'].map(preprocess)
    
    # Reading the each book description 
    for line in test_processed_docs:
        avgword2vec = None
        count = 0
        for word in line:
            if word in w2v_model.wv:
                count += 1
                if avgword2vec is None:
                    avgword2vec = w2v_model.wv[word]
                else:
                    avgword2vec = avgword2vec + w2v_model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings_test.append(avgword2vec)
        else:
            word_embeddings_test.append(np.array([0]*50, dtype='float32'))

In [ ]:
vectors_test()#train_df
# The Top 50 similar matches
def similarity(title):
    
    # finding cosine similarity for the vectors
    cosine_similarities = cosine_similarity(word_embeddings_test, word_embeddings_test)

    cntx = test_df[['title', 'posting_id']]
    #Reverse mapping of the index
    indices = pd.Series(test_df.index, index = test_df['title']).drop_duplicates()
         
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    uplimit = len(test_df['title']) if len(test_df['title']) < 50 else 50
    sim_scores = sim_scores[0:uplimit]
    post_indices = [i[0] for i in sim_scores if i[1] > .9]
    recommend = cntx.iloc[post_indices]
    output=[]
    output.append(cntx.iloc[idx,1])
    for index, row in recommend.iterrows():
        output.append(row['posting_id'])

    return ' '.join( np.unique(output))
    

In [ ]:
test_df['matches']=test_df['title'].apply(similarity)
    
test_df[['posting_id', 'matches']].to_csv('submission.csv', index = False)

In [ ]:
test_df[['posting_id', 'matches']].head()

# keep learning and enrich your intuition.Don't forgot to upvote !! :)